# Group Assignment QF627

*Group Members:*
* Anna Germaine Lim
* Peng Cheng
* Zenith Tay
* Gregory Tan

# Packages Used in This Workbook

In [68]:
## Data Download

import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

## For Visualisation

# Useful Functions Used in this Sheet

### Download of Data

In [69]:
def download_data (ticker,
                   start_date: str | datetime,
                   end_date: str | datetime) -> pd.DataFrame:
    data =\
    (
        yf.download(tickers = ticker,
                    start = start_date,
                    end = end_date
                    )
    )

    return data

In [70]:
# preallocate empty array and assign slice by chrisaycock

def np_shift(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

### Mean Reversion Strategies

In [71]:
def bollinger_band(price_data: pd.Series,
                            window: int = 14,
                                                ) -> pd.Series:

    price = price_data[price_col]

    std_dev_series =\
    (
        price
        .rolling(window = window)
        .std()
    )

    price_high =\
    (
        price + 2*std_dev_series
    )

    price_low =\
    (
        price - 2*std_dev_series
    )

    return price_high, price_low

### Momentum

In [72]:
## Moving Average

def generate_moving_avg(price_data: pd.Series,
                        window: int
                                              ) -> pd.Series:
    
    ma_series =\
    (
        pd.Series
        (   
            price_data
            .rolling(window = window)
            .mean(),

            name = 'MA' + str(window)
        )
    )

    return ma_series

In [73]:
## Exponential Moving Average

def generate_EMA(price_data: pd.Series, 
                    window: int
                            ) -> pd.Series:
    EMA = pd.Series(price_data
                    .ewm(span = window,
                         min_periods = window)
                    .mean(),
                    name = "EMA_" + str(window)
                    )
    return EMA

In [74]:
def generate_moving_avg_cross_signal(long_ma: pd.Series,
                                    short_ma: pd.Series) -> pd.Series:
    
    ## Sanity Check
    if len(long_ma) != len(short_ma):
        print('MA series lengths not equal, please check')
        return

    ## Return Signals
    else:

        moving_avg_cross_positions = np.where(short_ma > long_ma, 1.0, 0.0)
        moving_avg_cross_positions = np.where(short_ma < long_ma, -1.0 , moving_avg_cross_positions)

        moving_avg_cross_signals = np.where(moving_avg_cross_positions - np_shift(moving_avg_cross_positions,1) > 0, 1, 0)
        moving_avg_cross_signals = np.where(moving_avg_cross_positions - np_shift(moving_avg_cross_positions, 1) < 0, -1, moving_avg_cross_signals)

        buy_or_sell = pd.DataFrame({'MA_Cross_Signal':moving_avg_cross_signals, 'MA_Cross_Position': moving_avg_cross_positions},
                                   index = long_ma.index
                                   )

        return buy_or_sell

In [75]:
## Rate of Change

def generate_rate_of_change(price_data: pd.Series,
                            n: int
                            ) -> pd.Series:
    
    ROC = pd.Series(
                        (price_data - price_data.diff(n)) / price_data.diff(n),
                        name = 'ROC'+str(n),
                        # index = price_data.index
                    )
    
    return ROC

In [76]:
def generate_rate_of_change_signal(roc_data: pd.Series) -> pd.Series:

    roc_position = pd.Series(np.where(roc_data > 0, 1.0, 0.0), index=roc_data.index, name = 'ROC_Position')
    roc_signal = roc_position.diff()
    roc_signal.name = 'ROC_Signal'
    
    # roc_signal = roc_position - np_shift(roc_position, 1)


    # buy_or_sell = pd.DataFrame({'ROC_Position': roc_position, 'ROC_Signal': roc_signal},
    #                         #    index = roc_data.index
    #                            )

    return pd.concat([roc_position, roc_signal], axis=1)

In [77]:
## RSI

def generate_RSI(series, period):
    
    delta = series.diff().dropna()
    
    u = delta * 0
    d = u.copy()
    
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    
    u[u.index[period - 1]] = np.mean( u[:period] ) # 
    
    u = u.drop(u.index[:(period - 1)
                      ]
              )
    
    d[d.index[period - 1]] = np.mean( d[:period] )
    
    d = d.drop(d.index[:(period - 1)
                      ]
              )
    
    rs = u.ewm(com = period - 1, adjust = False).mean() / \
         d.ewm(com = period - 1, adjust = False).mean()
    
    return 100 - 100 / (1 + rs)

In [78]:
def generate_rsi_signal(rsi_data: pd.Series) -> pd.Series:

    rsi_position = pd.Series(np.where(rsi_data > 50, 1.0, -1.0), 
                             index= rsi_data.index, 
                             name = 'rsi_position')
    
    rsi_signal = rsi_position.diff()
    rsi_signal.name = 'rsi_signal'
    
    # rsi_signal = rsi_position - np_shift(rsi_position, 1)


    # buy_or_sell = pd.DataFrame({'rsi_position': rsi_position, 'rsi_signal': rsi_signal},
    #                         #    index = rsi_data.index
    #                            )

    return pd.concat([rsi_position, rsi_signal], axis=1)

In [79]:
def generate_force_index(data_price: pd.Series, data_vol: pd.Series, period: int) -> pd.Series:

    indicator = data_price.diff() * data_vol
    lag_indicator = indicator.ewm(span = period, adjust = False).mean()

    return lag_indicator

In [80]:
def generate_force_index_signal(data: pd.Series) -> pd.Series:

    position = pd.Series(np.where(data >= 0, 1.0, -1.0),
                        index = data.index,
                        name = 'FI_position'
                        )
    signal = position.diff()/2
    signal.name = 'FI_signal'

    return pd.concat([position, signal], axis = 1)

## Performance Metrics

In [81]:
## Annualised Sharpe

def annual_sharpe(returns):
    days = (returns.index[-1] - returns.index[0]).days
    
    return\
    (
        (
            (1+returns).prod()
            **(365/days) 
            - 1
        )
        /
        returns.std()
        /
        np.sqrt(252)
    )

In [82]:
## CAGR

def cagr(returns: pd.Series) -> float:
    days = (returns.index[-1] - returns.index[0]).days
    return ( (1 + returns).prod() )**(365/days) - 1   

In [83]:
### Max Drawdown

def max_drawdown(cumulative_returns):
    max_performance = cumulative_returns.cummax()
    dd = ((max_performance - cumulative_returns) / max_performance).max()
    return dd


### Longest Drawdown

def calculate_longest_drawdown(cumulative_returns):
    drawdown = cumulative_returns.cummax() - cumulative_returns
    period =\
    (
        np
        .diff(np
              .append(drawdown[drawdown == 0].index, 
                      drawdown.index[-1: ]
                    )
            )
    )
    return period.max() / np.timedelta64(1, "D")

In [84]:
def evaluate_returns(returns_series: pd.Series, to_print: bool = False):
    
    cum_returns_series = (1 + returns_series).cumprod()

    tot_returns = (1 + returns_series).prod() - 1
    CAGR = cagr(returns_series)
    Annualised_Sharpe = annual_sharpe(returns_series)
    Max_DD = max_drawdown(cum_returns_series)
    Longest_DD = calculate_longest_drawdown(cum_returns_series)

    if to_print == True:
      print('-- Summary of Returns -- \n',
            f'Total Returns: {tot_returns: .2%} \n',
            f'CAGR: {CAGR: .2%} \n',
            f'Annualised_Sharpe: {Annualised_Sharpe: .2%} \n',
            f'Max Drawdown: {Max_DD: .2%} \n',
            f'Longest Drawdown (Days): {Longest_DD}'            
            )

    return pd.Series([tot_returns, CAGR, Annualised_Sharpe, Max_DD, Longest_DD])

# Analysis

## Download Dataset

In [85]:
train_proportion = 0.75

spy_data =\
(
    download_data('SPY',
                  start_date = '2006-11-01',
                  end_date = '2025-11-12')
    .droplevel(level = 1,
               axis = 1)
    [['Close', 'Volume']]
).resample('W-FRI').agg({'Close': 'last', 'Volume': 'sum'})

spy_train_data = spy_data[:int(train_proportion*len(spy_data))]

spy_data_close = spy_train_data['Close'].to_frame()
spy_data_returns = spy_train_data['Close'].pct_change().to_frame().rename(columns= {'Close': 'Returns'})

C:\Users\TanFamily4\AppData\Local\Temp\ipykernel_15372\1474058220.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf.download(tickers = ticker,
[*********************100%***********************]  1 of 1 completed


In [86]:
spy_data_returns

,Returns
Date,
2006-11-03,NaN
2006-11-10,0.012451
2006-11-17,0.015770
2006-11-24,-0.000499
2006-12-01,-0.000926
...,...
2021-01-08,0.019739
2021-01-15,-0.014583
2021-01-22,0.019111


In [87]:
evaluate_returns(spy_data_returns['Returns'], to_print=True)

-- Summary of Returns -- 
 Total Returns:  279.10% 
 CAGR:  9.79% 
 Annualised_Sharpe:  23.57% 
 Max Drawdown:  54.61% 
 Longest Drawdown (Days): 1771.0


0       2.791024
1       0.097898
2       0.235731
3       0.546130
4    1771.000000
dtype: float64

## Momentum Strategies

In [88]:
time_periods = np.arange(10, 201, 10).tolist()

#### FI

In [89]:
def generate_fi_Metrics(time_periods):

    Metrics = pd.DataFrame()

    for i in time_periods:

        data = spy_data_returns.copy()
        
        data[f'fi{i}'] = generate_force_index(spy_data['Close'], spy_data['Volume'], i)
        
        data = pd.concat([data, generate_force_index_signal(data[f'fi{i}'])], axis = 1)
        
        data['Strat_returns'] = data['Returns'] * data['FI_position'].shift(1)
        
        # print(f'===Data for ROC{i}===')
        strat_series = evaluate_returns(data['Strat_returns'])
        strat_series.name = f'FI{i}'
        
        Metrics = pd.concat([Metrics, strat_series], axis = 1)

    Metrics.index = ['Total Returns', 'CAGR', 'Annualised Sharpe', 'Max Drawdown', 'Longest Drawdown (Days)']
    return Metrics.T

In [90]:
generate_fi_Metrics(list(range(10,251,10))).sort_values(by = 'Annualised Sharpe', ascending=False)

,Total Returns,CAGR,Annualised Sharpe,Max Drawdown,Longest Drawdown (Days)
FI10,-0.374805,-0.032382,-0.077719,0.484255,4354.0
FI40,-0.551478,-0.054644,-0.131191,0.725767,4354.0
FI20,-0.552316,-0.054768,-0.131489,0.722403,4354.0
FI50,-0.572262,-0.057782,-0.138736,0.751453,4354.0
FI30,-0.586335,-0.059989,-0.144042,0.715841,4354.0
FI60,-0.601474,-0.062442,-0.149941,0.768427,4354.0
FI70,-0.601662,-0.062473,-0.150016,0.749208,4354.0
FI80,-0.669855,-0.074730,-0.179515,0.796985,4354.0
FI110,-0.672303,-0.075212,-0.180677,0.802884,4354.0
FI100,-0.673990,-0.075547,-0.181483,0.803711,4354.0


In [91]:
type(spy_data_close)

pandas.core.frame.DataFrame

In [92]:
def generate_ROC_Metrics(time_periods):

    ROC_Metrics = pd.DataFrame()

    for i in time_periods:
        roc_data = spy_data_returns.copy()
        
        roc_data[f'ROC{i}'] = generate_rate_of_change(spy_data_close['Close'], i)
        roc_data = pd.concat([roc_data, generate_rate_of_change_signal(roc_data[f'ROC{i}'])], axis = 1)
        roc_data['Strat_returns'] = roc_data['Returns'] * roc_data['ROC_Position'].shift(1)
        
        # print(f'===Data for ROC{i}===')
        roc_series = evaluate_returns(roc_data['Strat_returns'])
        roc_series.name = f'ROC{i}'
        
        ROC_Metrics = pd.concat([ROC_Metrics, roc_series], axis = 1)

    ROC_Metrics.index = ['Total Returns', 'CAGR', 'Annualised Sharpe', 'Max Drawdown', 'Longest Drawdown (Days)']
    return ROC_Metrics.T

In [93]:
generate_ROC_Metrics(time_periods).sort_values(by = 'Annualised Sharpe', ascending=False)

,Total Returns,CAGR,Annualised Sharpe,Max Drawdown,Longest Drawdown (Days)
ROC100,2.546123,0.092771,0.321478,0.318290,357.0
ROC70,2.541795,0.092678,0.321278,0.202251,714.0
ROC170,2.467795,0.091062,0.320583,0.318290,315.0
ROC150,2.704475,0.096122,0.319853,0.318290,315.0
ROC190,2.363653,0.088733,0.319186,0.318290,315.0
ROC200,2.302022,0.087323,0.315491,0.318290,427.0
ROC90,2.555621,0.092976,0.315145,0.318290,357.0
ROC80,2.403237,0.089626,0.310517,0.212966,938.0
ROC180,2.290074,0.087046,0.309823,0.318290,322.0
ROC130,2.400761,0.089570,0.303794,0.332358,315.0


In [94]:
def generate_RSI_Metrics(time_periods):

    Metrics = pd.DataFrame()

    for i in time_periods:
        data = spy_data_returns.copy()
        
        data[f'RSI{i}'] = generate_RSI(spy_data_close['Close'], i)
        
        data = pd.concat([data, generate_rsi_signal(data[f'RSI{i}'])], axis = 1)
        
        data['Strat_returns'] = data['Returns'] * data['rsi_position'].shift(1)
        
        # print(f'===Data for ROC{i}===')
        strat_series = evaluate_returns(data['Strat_returns'])
        strat_series.name = f'RSI{i}'
        
        Metrics = pd.concat([Metrics, strat_series], axis = 1)

    Metrics.index = ['Total Returns', 'CAGR', 'Annualised Sharpe', 'Max Drawdown', 'Longest Drawdown (Days)']
    return Metrics.T

In [95]:
generate_RSI_Metrics(range(10, 252, 10)).sort_values(by = 'Annualised Sharpe', ascending = False)

,Total Returns,CAGR,Annualised Sharpe,Max Drawdown,Longest Drawdown (Days)
RSI180,1.674250,0.071372,0.171632,0.559958,3073.0
RSI250,1.481152,0.065759,0.158098,0.578603,3199.0
RSI190,1.471127,0.065457,0.157369,0.580306,3206.0
RSI200,1.397000,0.063185,0.151893,0.592896,3227.0
RSI210,1.347380,0.061627,0.148140,0.601323,3241.0
RSI220,1.246126,0.058352,0.140248,0.618520,3465.0
RSI240,1.243047,0.058250,0.140003,0.619043,3465.0
RSI80,1.079752,0.052659,0.126539,0.574706,3010.0
RSI230,1.063855,0.052093,0.125176,0.649477,3899.0
RSI170,0.934571,0.047334,0.113722,0.604903,3227.0


In [96]:
MA_list =\
(
    list(range(10, 201, 10))
) 

In [97]:
def generate_ema_cross_metrics(short_ma: list[int], long_ma: list[int]) -> pd.Series:
    
    ma_cross_data = spy_data_close.copy()
    ma_cross_summary_table = pd.DataFrame()

    for j in long_ma:

        for i in short_ma:

            if i < j:
            
                short_ma_cross_data = generate_EMA(ma_cross_data['Close'], i)             
                long_ma_cross_data = generate_EMA(ma_cross_data['Close'], j)

                ma_cross_returns =\
                (
                    pd
                    .concat(
                            (spy_data_returns, generate_moving_avg_cross_signal(long_ma_cross_data, short_ma_cross_data)),
                            axis = 1
                            )
                )

                ma_cross_returns['Strat_Returns'] = ma_cross_returns['Returns']*ma_cross_returns['MA_Cross_Position'].shift(1)

                ma_cross_summary_stat_series = evaluate_returns(ma_cross_returns['Strat_Returns'])
                ma_cross_summary_stat_series.name = f'EMA{i} + EMA{j}'


                ma_cross_summary_table = pd.concat([ma_cross_summary_table, ma_cross_summary_stat_series], axis = 1)

    ma_cross_summary_table.index = ['Total Returns', 'CAGR', 'Annualised Sharpe', 'Max Drawdown', 'Longest Drawdown (Days)']
    return ma_cross_summary_table.T

In [98]:
generate_ema_cross_metrics(MA_list, MA_list).sort_values(by = 'Annualised Sharpe', ascending=False)

,Total Returns,CAGR,Annualised Sharpe,Max Drawdown,Longest Drawdown (Days)
EMA20 + EMA190,3.279346,0.107260,0.346182,0.318290,315.0
EMA20 + EMA200,3.105237,0.104042,0.341948,0.318290,315.0
EMA40 + EMA180,3.178068,0.105403,0.334738,0.318290,315.0
EMA10 + EMA200,2.988190,0.101806,0.334526,0.318290,315.0
EMA30 + EMA200,2.988190,0.101806,0.334526,0.318290,315.0
...,...,...,...,...,...
EMA10 + EMA40,0.294047,0.018230,0.044267,0.554401,1715.0
EMA20 + EMA50,0.147756,0.009705,0.023617,0.490311,3136.0
EMA10 + EMA30,0.095525,0.006415,0.015499,0.554401,2177.0
EMA10 + EMA60,0.020466,0.001421,0.003480,0.603809,1946.0


In [99]:
def generate_ma_cross_metrics(short_ma: list[int], long_ma: list[int]) -> pd.Series:
    
    ma_cross_data = spy_data_close.copy()

    ma_cross_summary_table = pd.DataFrame()

    for j in long_ma:

        for i in short_ma:

            if i < j:
            
                short_ma_cross_data = generate_moving_avg(ma_cross_data['Close'], i)
                long_ma_cross_data = generate_moving_avg(ma_cross_data['Close'], j)

                ma_cross_returns =\
                (
                    pd
                    .concat(
                            (spy_data_returns, generate_moving_avg_cross_signal(long_ma_cross_data, short_ma_cross_data)),
                            axis = 1
                            )
                )

                ma_cross_returns['Strat_Returns'] = ma_cross_returns['Returns']*ma_cross_returns['MA_Cross_Position'].shift(1)

                ma_cross_summary_stat_series = evaluate_returns(ma_cross_returns['Strat_Returns'])
                ma_cross_summary_stat_series.name = f'MA{i} + MA{j}'


                ma_cross_summary_table = pd.concat([ma_cross_summary_table, ma_cross_summary_stat_series], axis = 1)

    ma_cross_summary_table.index = ['Total Returns', 'CAGR', 'Annualised Sharpe', 'Max Drawdown', 'Longest Drawdown (Days)']
    return ma_cross_summary_table.T, ma_cross_returns

In [100]:
ma_cross_summary = generate_ma_cross_metrics(MA_list, MA_list)[0]
ma_cross_summary.sort_values(by = 'Total Returns', ascending = False).head(10)

,Total Returns,CAGR,Annualised Sharpe,Max Drawdown,Longest Drawdown (Days)
MA40 + MA90,3.466234,0.110582,0.278230,0.437183,1589.0
MA40 + MA160,3.349125,0.108516,0.341281,0.318290,315.0
MA40 + MA150,3.094130,0.103832,0.322501,0.318290,385.0
MA30 + MA150,3.063531,0.103252,0.320682,0.318290,399.0
MA50 + MA70,3.056251,0.103113,0.256137,0.464209,1715.0
MA50 + MA150,3.055236,0.103094,0.320187,0.318290,399.0
MA30 + MA100,2.999716,0.102029,0.257376,0.443429,1687.0
MA30 + MA160,2.986586,0.101775,0.319881,0.318290,315.0
MA140 + MA180,2.979287,0.101633,0.322653,0.318290,581.0
MA50 + MA160,2.925165,0.100576,0.316081,0.318290,315.0


## Test Dataset

In [125]:
row = 90

spy_test_data = spy_data[int(train_proportion*len(spy_data)-row):]

spy_data_close = spy_test_data['Close'].to_frame()
spy_data_returns = spy_test_data['Close'].pct_change().to_frame().rename(columns= {'Close': 'Returns'})

display(spy_test_data[row:])

evaluate_returns(spy_data_returns.loc[spy_data_returns.index[row:],'Returns'], to_print=True);

Price,Close,Volume
Date,,
2021-02-12,367.603149,226577300
2021-02-19,365.159576,246216800
2021-02-26,356.106232,546504300
2021-03-05,359.167725,640356900
2021-03-12,368.932648,497580800
...,...,...
2025-10-17,664.390015,457106900
2025-10-24,677.250000,337267400
2025-10-31,682.059998,374234900


-- Summary of Returns -- 
 Total Returns:  88.16% 
 CAGR:  14.21% 
 Annualised_Sharpe:  39.88% 
 Max Drawdown:  23.93% 
 Longest Drawdown (Days): 714.0


In [103]:
generate_ma_cross_metrics([40], [90])[0]

,Total Returns,CAGR,Annualised Sharpe,Max Drawdown,Longest Drawdown (Days)
MA40 + MA90,0.33784,0.063107,0.237668,0.240572,588.0


In [104]:
generate_ma_cross_metrics([40], [160])[0]

,Total Returns,CAGR,Annualised Sharpe,Max Drawdown,Longest Drawdown (Days)
MA40 + MA160,0.360676,0.066897,0.32269,0.168772,133.0


In [105]:
evaluate_returns(generate_ma_cross_metrics([40], [160])[1]['Strat_Returns'], to_print=True)

-- Summary of Returns -- 
 Total Returns:  36.07% 
 CAGR:  6.69% 
 Annualised_Sharpe:  32.27% 
 Max Drawdown:  16.88% 
 Longest Drawdown (Days): 133.0


0      0.360676
1      0.066897
2      0.322690
3      0.168772
4    133.000000
dtype: float64

In [106]:
generate_ma_cross_metrics([40], [150])[0]

,Total Returns,CAGR,Annualised Sharpe,Max Drawdown,Longest Drawdown (Days)
MA40 + MA150,0.473287,0.084883,0.402817,0.168772,133.0


In [107]:
print('== Strategy Return of MA ==')
evaluate_returns(generate_ma_cross_metrics([40], [160])[1]['Strat_Returns'], to_print=True);

print('\n')

print('== Benchmark Returns ==')
evaluate_returns(spy_data_returns.loc[spy_data_returns.index[160:],'Returns'], to_print=True);

== Strategy Return of MA ==
-- Summary of Returns -- 
 Total Returns:  36.07% 
 CAGR:  6.69% 
 Annualised_Sharpe:  32.27% 
 Max Drawdown:  16.88% 
 Longest Drawdown (Days): 133.0


== Benchmark Returns ==
-- Summary of Returns -- 
 Total Returns:  36.07% 
 CAGR:  20.02% 
 Annualised_Sharpe:  58.18% 
 Max Drawdown:  16.88% 
 Longest Drawdown (Days): 133.0
